In [83]:
import pandas as pd
import polars as pl
import numpy as np
from openai import OpenAI
from tqdm import tqdm
import re

In [48]:
df = pd.read_parquet("/data/gbg141/Arlequin/data/Arlequin_twitter_MAGA.parquet")
df

,id,content,no. of comments,no. of likes,no. of shares,no. of views,post subtype,language,date,username,bio,no. of followers,no. of follows,reply_or_retweet
0,0_150937_8_1959768220274929930,"23. endless paranoid lunacy, now we have the c...",0,1,1,1103,reply,en,2025-08-25,austerewyatt12,Scientist \ Based Inventor \ Nuclear Physisist...,165500,403,None
1,0_150937_8_1959768262759288951,@GavinNewsom You've gerrymandered 43 out of 52...,0,27,6,1056,reply,en,2025-08-25,MAGAResource,MAGA News 24/7. Click 🔔+ Notify Above To Get B...,216682,980,GavinNewsom
2,0_150937_8_1959768489360826693,@GuntherEagleman Feels so good to be back. Ame...,0,100,12,3374,reply,en,2025-08-25,MAGAVoice,Want to take back OUR country? JOIN ME 🇺🇸 Dona...,1291901,518,GuntherEagleman
3,0_150937_8_1959768526048444825,@MAGAPosts I absolutely did,0,28,2,1393,reply,en,2025-08-25,MAGAVoice,Want to take back OUR country? JOIN ME 🇺🇸 Dona...,1291901,518,MAGAPosts
4,0_150937_8_1959768821293613507,JAG Corps ACTIVATE! https://t.co/673nVG0hWO,0,36,5,1152,quote,en,2025-08-25,paintsaints,All American Girl / Child of God / Believer / ...,36824,5315,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39074,0_150937_8_1967556303355846705,More on Charlie Kirk's legacyhttps://t.co/z8rB...,0,225,40,30820,original post,en,2025-09-15,marklevinshow,THIS IS THE OFFICIAL MARK LEVIN SHOW X PAGE. \...,4907838,5,None
39075,0_150937_8_1967556408964186255,GM fam! How's everyone this morning? https://t...,0,203,32,9530,original post,en,2025-09-15,MJTruthUltra,"Watchdog, Researcher, Humor Specialist Unshaka...",563666,2777,None
39076,0_150937_8_1967556713462264074,@TalkWithSally Not sure what you mean by movin...,0,3,0,559,reply,en,2025-09-15,MJTruthUltra,"Watchdog, Researcher, Humor Specialist Unshaka...",563666,2777,TalkWithSally
39077,0_150937_8_1967558427061997719,The Islamists on Charlie's assassinationhttps:...,0,209,67,30035,original post,en,2025-09-15,marklevinshow,THIS IS THE OFFICIAL MARK LEVIN SHOW X PAGE. \...,4907838,5,None


In [118]:
# generate integer message ids
df["id"] = pd.RangeIndex(len(df))

In [119]:
df

,id,content,no. of comments,no. of likes,no. of shares,no. of views,post subtype,language,date,username,bio,no. of followers,no. of follows,reply_or_retweet,user_id,reply_to_user_id,mentions_user_id,embedding,bio_embedding,datestamp
0,0,"23. endless paranoid lunacy, now we have the c...",0,1,1,1103,reply,en,2025-08-25,austerewyatt12,Scientist \ Based Inventor \ Nuclear Physisist...,165500,403,None,0,[],[],"[-0.018080048263072968, 0.029990049079060555, ...","[-0.002727521350607276, -0.003640637034550309,...",739488
1,1,@GavinNewsom You've gerrymandered 43 out of 52...,0,27,6,1056,reply,en,2025-08-25,MAGAResource,MAGA News 24/7. Click 🔔+ Notify Above To Get B...,216682,980,GavinNewsom,1,[],[],"[-0.03978833556175232, -0.01148630678653717, 0...","[-0.01618366502225399, -0.007729790173470974, ...",739488
2,2,@GuntherEagleman Feels so good to be back. Ame...,0,100,12,3374,reply,en,2025-08-25,MAGAVoice,Want to take back OUR country? JOIN ME 🇺🇸 Dona...,1291901,518,GuntherEagleman,2,[],[],"[-0.015221292153000832, 0.004700069781392813, ...","[-0.021167198196053505, 0.004752574488520622, ...",739488
3,3,@MAGAPosts I absolutely did,0,28,2,1393,reply,en,2025-08-25,MAGAVoice,Want to take back OUR country? JOIN ME 🇺🇸 Dona...,1291901,518,MAGAPosts,2,[],[],"[0.012352042831480503, 0.0012695561163127422, ...","[-0.021167198196053505, 0.004752574488520622, ...",739488
4,4,JAG Corps ACTIVATE! https://t.co/673nVG0hWO,0,36,5,1152,quote,en,2025-08-25,paintsaints,All American Girl / Child of God / Believer / ...,36824,5315,None,3,[],[],"[-0.03979841619729996, -0.009771346114575863, ...","[-0.026603348553180695, -0.023830875754356384,...",739488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39074,39074,More on Charlie Kirk's legacyhttps://t.co/z8rB...,0,225,40,30820,original post,en,2025-09-15,marklevinshow,THIS IS THE OFFICIAL MARK LEVIN SHOW X PAGE. \...,4907838,5,None,10,[],[],"[-0.0430118702352047, 0.02245650812983513, -0....","[-0.03571410849690437, -0.05290907248854637, -...",739509
39075,39075,GM fam! How's everyone this morning? https://t...,0,203,32,9530,original post,en,2025-09-15,MJTruthUltra,"Watchdog, Researcher, Humor Specialist Unshaka...",563666,2777,None,19,[],[],"[-0.036922164261341095, -0.017795013263821602,...","[-0.04067656770348549, -0.0028596846386790276,...",739509
39076,39076,@TalkWithSally Not sure what you mean by movin...,0,3,0,559,reply,en,2025-09-15,MJTruthUltra,"Watchdog, Researcher, Humor Specialist Unshaka...",563666,2777,TalkWithSally,19,[],[],"[-0.027035817503929138, 0.03753622993826866, -...","[-0.04067656770348549, -0.0028596846386790276,...",739509
39077,39077,The Islamists on Charlie's assassinationhttps:...,0,209,67,30035,original post,en,2025-09-15,marklevinshow,THIS IS THE OFFICIAL MARK LEVIN SHOW X PAGE. \...,4907838,5,None,10,[],[],"[0.009211130440235138, 0.028164757415652275, -...","[-0.03571410849690437, -0.05290907248854637, -...",739509


In [53]:
# generate integer user ids
user_mapping = {user: idx for idx, user in enumerate(df['username'].unique())}
df['user_id'] = df['username'].map(user_mapping)

In [116]:
# generate integer time stamps
df["datestamp"] = df["date"].apply(lambda x: x.toordinal())

In [114]:
df["date"].iloc[15000].toordinal()

739497

In [ ]:
# generate reply_to_user_id
# only include users that exist in the dataset
df['reply_to_user_id'] = df['reply_or_retweet'].apply(
    lambda x: [user_mapping[username] for username in (x if isinstance(x, list) else [x] if x is not None else []) if username in user_mapping]
)

In [81]:
# generate mentions_user_id
# only include users that exist in the dataset
def extract_mentions(content):
    """Extract @username mentions from content."""
    if pd.isna(content):
        return []
    # Find all @username patterns
    mentions = re.findall(r'@(\w+)', str(content))
    # Filter to only include users in user_mapping
    return [user_mapping[username] for username in mentions if username in user_mapping]

df['mentions_user_id'] = df['content'].apply(extract_mentions)

In [88]:
# message embeddings
# get messages
msgs = np.array(df['content'])

# create an OpenAI client with your deepinfra token and endpoint
client = OpenAI(
    api_key="7jIPsm1yv398SZpzLaE0qw2DIs2Y5CZG",
    base_url="https://api.deepinfra.com/v1/openai",
)

# create batches
batch_size = 512
batches = [msgs[i:i + batch_size] for i in range(0, len(msgs), batch_size)]

# embed
embeddings = []
for i, batch in enumerate(tqdm(batches, desc="Embedding batches")):
    embedding = client.embeddings.create(
        model='BAAI/bge-m3',
        input = batch.tolist(),
        dimensions=1024,
        encoding_format='float',
    )
    embeddings.extend([x.embedding for x in embedding.data])

# save
df['embedding'] = embeddings

Embedding batches: 100%|██████████| 77/77 [06:00<00:00,  4.68s/it]


In [91]:
# get messages
bios = np.array(df['bio'].replace({None: ""}))

# create batches
batch_size = 512
batches = [bios[i:i + batch_size] for i in range(0, len(bios), batch_size)]

# embed
embeddings = []
for i, batch in enumerate(tqdm(batches, desc="Embedding batches")):
    embedding = client.embeddings.create(
        model='BAAI/bge-m3',
        input=batch.tolist(),
        dimensions=1024,
        encoding_format='float',
    )
    embeddings.extend([x.embedding for x in embedding.data])

# save
df['bio_embedding'] = embeddings

Embedding batches: 100%|██████████| 77/77 [05:01<00:00,  3.91s/it]


In [120]:
# save
df.to_parquet("/data/gbg141/Arlequin/data/MAGA_embedded.parquet", index=False)

In [121]:
df_loaded = pd.read_parquet("/data/gbg141/Arlequin/data/MAGA_embedded.parquet")
df_loaded

,id,content,no. of comments,no. of likes,no. of shares,no. of views,post subtype,language,date,username,bio,no. of followers,no. of follows,reply_or_retweet,user_id,reply_to_user_id,mentions_user_id,embedding,bio_embedding,datestamp
0,0,"23. endless paranoid lunacy, now we have the c...",0,1,1,1103,reply,en,2025-08-25,austerewyatt12,Scientist \ Based Inventor \ Nuclear Physisist...,165500,403,None,0,[],[],"[-0.018080048263072968, 0.029990049079060555, ...","[-0.002727521350607276, -0.003640637034550309,...",739488
1,1,@GavinNewsom You've gerrymandered 43 out of 52...,0,27,6,1056,reply,en,2025-08-25,MAGAResource,MAGA News 24/7. Click 🔔+ Notify Above To Get B...,216682,980,GavinNewsom,1,[],[],"[-0.03978833556175232, -0.01148630678653717, 0...","[-0.01618366502225399, -0.007729790173470974, ...",739488
2,2,@GuntherEagleman Feels so good to be back. Ame...,0,100,12,3374,reply,en,2025-08-25,MAGAVoice,Want to take back OUR country? JOIN ME 🇺🇸 Dona...,1291901,518,GuntherEagleman,2,[],[],"[-0.015221292153000832, 0.004700069781392813, ...","[-0.021167198196053505, 0.004752574488520622, ...",739488
3,3,@MAGAPosts I absolutely did,0,28,2,1393,reply,en,2025-08-25,MAGAVoice,Want to take back OUR country? JOIN ME 🇺🇸 Dona...,1291901,518,MAGAPosts,2,[],[],"[0.012352042831480503, 0.0012695561163127422, ...","[-0.021167198196053505, 0.004752574488520622, ...",739488
4,4,JAG Corps ACTIVATE! https://t.co/673nVG0hWO,0,36,5,1152,quote,en,2025-08-25,paintsaints,All American Girl / Child of God / Believer / ...,36824,5315,None,3,[],[],"[-0.03979841619729996, -0.009771346114575863, ...","[-0.026603348553180695, -0.023830875754356384,...",739488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39074,39074,More on Charlie Kirk's legacyhttps://t.co/z8rB...,0,225,40,30820,original post,en,2025-09-15,marklevinshow,THIS IS THE OFFICIAL MARK LEVIN SHOW X PAGE. \...,4907838,5,None,10,[],[],"[-0.0430118702352047, 0.02245650812983513, -0....","[-0.03571410849690437, -0.05290907248854637, -...",739509
39075,39075,GM fam! How's everyone this morning? https://t...,0,203,32,9530,original post,en,2025-09-15,MJTruthUltra,"Watchdog, Researcher, Humor Specialist Unshaka...",563666,2777,None,19,[],[],"[-0.036922164261341095, -0.017795013263821602,...","[-0.04067656770348549, -0.0028596846386790276,...",739509
39076,39076,@TalkWithSally Not sure what you mean by movin...,0,3,0,559,reply,en,2025-09-15,MJTruthUltra,"Watchdog, Researcher, Humor Specialist Unshaka...",563666,2777,TalkWithSally,19,[],[],"[-0.027035817503929138, 0.03753622993826866, -...","[-0.04067656770348549, -0.0028596846386790276,...",739509
39077,39077,The Islamists on Charlie's assassinationhttps:...,0,209,67,30035,original post,en,2025-09-15,marklevinshow,THIS IS THE OFFICIAL MARK LEVIN SHOW X PAGE. \...,4907838,5,None,10,[],[],"[0.009211130440235138, 0.028164757415652275, -...","[-0.03571410849690437, -0.05290907248854637, -...",739509


In [99]:
df["id"].iloc[0] == df["id"].iloc[5]

np.False_

In [78]:
user_mapping

{'austerewyatt12': 0,
 'MAGAResource': 1,
 'MAGAVoice': 2,
 'paintsaints': 3,
 'magaforce24': 4,
 'SirCensorLot': 5,
 'davereaboi': 6,
 'SpunkyPatriot_': 7,
 'Twiztid_One_': 8,
 'LauraLoomer': 9,
 'marklevinshow': 10,
 'JackPosobiec': 11,
 'texan_maga': 12,
 'GolfingFrogs': 13,
 'charliekirk11': 14,
 'tedcruz': 15,
 'truth_ac': 16,
 'MattWalshBlog': 17,
 'THeinrich22': 18,
 'MJTruthUltra': 19,
 'JtShaggy': 20,
 'seanhannity': 21,
 'RepMTG': 22,
 'mtgreenee': 23,
 'TuckerCarlson': 24,
 'DC_Draino': 25,
 'LindseyGrahamSC': 26,
 'RealSeq16': 27,
 'JDVance': 28,
 'RealCandaceO': 29,
 'nancy_hamm1': 30,
 'Badged_Patriot': 31,
 'JamesOKeefeIII': 32,
 '1776Eagles': 33,
 'wwwMAGA_': 34,
 'laurenboebert': 35,
 'kayleighmcenany': 36,
 'TulsiGabbard': 37,
 'realDonaldTrump': 38}

In [74]:
df.iloc[39065]

id                                         15093781967551033623945685
content             RT @robertdunlap947: The media and talking hea...
no. of comments                                                     0
no. of likes                                                        0
no. of shares                                                       0
no. of views                                                        0
post subtype                                                   repost
language                                                           en
date                                                       2025-09-15
username                                                  THeinrich22
bio                 🎺 TRUMP💥MAGA, Nationalist, #Patriot, #Constitu...
no. of followers                                                70857
no. of follows                                                  53446
reply_or_retweet                                                 None
user_id             

In [75]:
df

,id,content,no. of comments,no. of likes,no. of shares,no. of views,post subtype,language,date,username,bio,no. of followers,no. of follows,reply_or_retweet,user_id,reply_to_user_id
0,15093781959768220274929930,"23. endless paranoid lunacy, now we have the c...",0,1,1,1103,reply,en,2025-08-25,austerewyatt12,Scientist \ Based Inventor \ Nuclear Physisist...,165500,403,None,0,[]
1,15093781959768262759288951,@GavinNewsom You've gerrymandered 43 out of 52...,0,27,6,1056,reply,en,2025-08-25,MAGAResource,MAGA News 24/7. Click 🔔+ Notify Above To Get B...,216682,980,GavinNewsom,1,[]
2,15093781959768489360826693,@GuntherEagleman Feels so good to be back. Ame...,0,100,12,3374,reply,en,2025-08-25,MAGAVoice,Want to take back OUR country? JOIN ME 🇺🇸 Dona...,1291901,518,GuntherEagleman,2,[]
3,15093781959768526048444825,@MAGAPosts I absolutely did,0,28,2,1393,reply,en,2025-08-25,MAGAVoice,Want to take back OUR country? JOIN ME 🇺🇸 Dona...,1291901,518,MAGAPosts,2,[]
4,15093781959768821293613507,JAG Corps ACTIVATE! https://t.co/673nVG0hWO,0,36,5,1152,quote,en,2025-08-25,paintsaints,All American Girl / Child of God / Believer / ...,36824,5315,None,3,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39074,15093781967556303355846705,More on Charlie Kirk's legacyhttps://t.co/z8rB...,0,225,40,30820,original post,en,2025-09-15,marklevinshow,THIS IS THE OFFICIAL MARK LEVIN SHOW X PAGE. \...,4907838,5,None,10,[]
39075,15093781967556408964186255,GM fam! How's everyone this morning? https://t...,0,203,32,9530,original post,en,2025-09-15,MJTruthUltra,"Watchdog, Researcher, Humor Specialist Unshaka...",563666,2777,None,19,[]
39076,15093781967556713462264074,@TalkWithSally Not sure what you mean by movin...,0,3,0,559,reply,en,2025-09-15,MJTruthUltra,"Watchdog, Researcher, Humor Specialist Unshaka...",563666,2777,TalkWithSally,19,[]
39077,15093781967558427061997719,The Islamists on Charlie's assassinationhttps:...,0,209,67,30035,original post,en,2025-09-15,marklevinshow,THIS IS THE OFFICIAL MARK LEVIN SHOW X PAGE. \...,4907838,5,None,10,[]


In [77]:
np.unique(df["reply_to_user_id"].replace({None: ""}))

array([list([]), list([1]), list([2]), list([3]), list([5]), list([8]),
       list([9]), list([10]), list([11]), list([13]), list([14]),
       list([15]), list([17]), list([19]), list([22]), list([25]),
       list([26]), list([28]), list([29]), list([30]), list([32]),
       list([37]), list([38])], dtype=object)

In [4]:
np.unique(df["reply_or_retweet"].replace({None: "None"}))

array(['01splcheck', '0HOUR1__', '0tt0rama', ..., 'zierbox', 'zk__nova',
       'zriboua'], shape=(2407,), dtype=object)

In [5]:
"GavinNewsom" in np.unique(df["reply_or_retweet"].replace({None: "None"}))

True

In [38]:
df['content'][39078]

'FBI Director Kash Patel confirms DNA match of Tyler Robinson with the towel used to wrap the rifle and a screwdriver found at the sniper nestATF is analyzing the rifle itself and will provide DNA results soonKash confirms dozens of people in Leftist group chats are being https://t.co/sxTx26b9Yc'

In [ ]:
# Remove URLs from the content column and replace by a tag
# df['content'].apply(lambda x: re.sub(r'http\S+|www\.\S+', ' [url link]', str(x)))[39074]

"More on Charlie Kirk's legacy [url link]"

In [ ]:
# Remove URLs and user mentions from content and replace by a tag
# df['content'].apply(lambda x: re.sub(r'@\w+', '[user mention]', str(x)))[3]

'[user mention] I absolutely did'

In [12]:
# get messages
msgs = np.array(df['content'])[:10000]

# create an OpenAI client with your deepinfra token and endpoint
client = OpenAI(
    api_key="7jIPsm1yv398SZpzLaE0qw2DIs2Y5CZG",
    base_url="https://api.deepinfra.com/v1/openai",
)

# create batches
batch_size = 512
batches = np.array_split(msgs, int(len(msgs) / batch_size))

# embed
embeddings = []
for i, batch in enumerate(batches):
    embedding = client.embeddings.create(
        model='BAAI/bge-m3',
        input = batch,
        dimensions=1024,
        encoding_format='float',
    )
    embeddings.extend([x.embedding for x in embedding.data])
    print('Batch ', i+1, ' Done')

# save
df['embedding'] = embeddings

Batch  1  Done
Batch  2  Done
Batch  3  Done
Batch  4  Done
Batch  5  Done
Batch  6  Done
Batch  7  Done
Batch  8  Done
Batch  9  Done
Batch  10  Done
Batch  11  Done
Batch  12  Done
Batch  13  Done
Batch  14  Done
Batch  15  Done
Batch  16  Done
Batch  17  Done
Batch  18  Done
Batch  19  Done


ValueError: Length of values (10000) does not match length of index (39079)

In [13]:
len(embeddings)

10000

In [31]:
# get messages
bios = np.array(df['bio'].replace({None: ""}))[:1000]

# create batches
batch_size = 512
batches = [bios[i:i + batch_size] for i in range(0, len(bios), batch_size)]

# embed
embeddings = []
for i, batch in enumerate(batches):
    embedding = client.embeddings.create(
        model='BAAI/bge-m3',
        input=batch.tolist(),
        dimensions=1024,
        encoding_format='float',
    )
    embeddings.extend([x.embedding for x in embedding.data])
    print('Batch ', i+1, ' Done')

# save
df['bio_embedding'] = embeddings

Batch  1  Done
Batch  2  Done


ValueError: Length of values (1000) does not match length of index (39079)

In [33]:
batches[1].shape

(488,)

In [24]:
np.unique(df["bio"].replace({None: ""})).shape

(39,)

In [41]:
twitter = pd.read_parquet("/data/gbg141/Arlequin/data/twitter_s10k.parquet")

In [122]:
twitter

,id,message,user_id,mentions_user_id,reply_to_user_id,embedding
0,1.885405e+18,"""@valerieboyer13 @leJDD Le JDD il ne manquait ...",1603103689585135616,"['276934698', '49678509']",['276934698'],"[-0.0011729525867849588, 0.03560536354780197, ..."
1,1.881385e+18,"""@AjaxVV8 En vervanger? jullie idee ik zou het...",54851208,['1134847436164349952'],['1134847436164349952'],"[-0.027059949934482574, -0.008414551615715027,..."
2,1.883763e+18,"""@Spidermandurct Notre équipe ""type"" montre be...",1348259857460326405,['1515443772338884616'],['1515443772338884616'],"[-0.06197907030582428, -0.0012791703920811415,..."
3,1.880636e+18,"""@christinblack_ Awwwnnnnn....Hot 🔥 my Sweet L...",2774381316,['1543888212149534720'],['1543888212149534720'],"[0.014438987709581852, 0.019704950973391533, -..."
4,1.880621e+18,"""@MiltysWorldX Ah!",1406516521380220930,['1259314706277416962'],['1259314706277416962'],"[-0.028360256925225258, -0.001895296503789723,..."
...,...,...,...,...,...,...
9995,1.883012e+18,"""@SavvyRinu Very very sad",882321415,['991043283534139392'],['991043283534139392'],"[-0.03760543838143349, 0.02799161709845066, -0..."
9996,1.881306e+18,"""@realTrumpNewsX https://t.co/arH1JchveU @SHAR...",1591005173068988416,['1810341133911994368'],['1810341133911994368'],"[-0.03130640462040901, -0.014854795299470425, ..."
9997,1.881112e+18,"""@MashiRafael https://t.co/5qafWV8XBM @SHARE E...",1318196918829391877,['209780362'],['209780362'],"[-0.016084255650639534, 0.020587023347616196, ..."
9998,1.885315e+18,"""@TMerk4343 @mjfree He's one of the most disgu...",48079936,"['1589295407632420865', '24113137']",['1589295407632420865'],"[-0.035898495465517044, 0.023607270792126656, ..."


In [40]:
pd.read_csv("/data/gbg141/Arlequin/data/twitter_dataset.csv")

,id,conversation_id,source,message,user_id,username,name,mentions_user_id,mentions_name,mentions_username,reply_to_user_id,reply_to_username,anger,joy,disgust,fear,neutral,sadness,surprise
0,1.885383e+18,1.885378e+18,twitter,"""@roberthabeck Lächerliches Statement \nDie Gr...",1393649084020428801,Marcel66036691,Marcel,['1852291228475289600'],['Robert Habeck'],['roberthabeck'],['1852291228475289600'],['roberthabeck'],0.01,0.00,0.01,0.01,0.00,0.97,0.00
1,1.885383e+18,1.885373e+18,twitter,"""@FabrizioRomano 😂😂😂 Spurs are a joke",3076467441,FredMarcel09,FredMarcel09,['330262748'],['Fabrizio Romano'],['FabrizioRomano'],['330262748'],['FabrizioRomano'],0.35,0.31,0.09,0.04,0.05,0.05,0.11
2,1.885380e+18,1.885359e+18,twitter,"""@Zebandrew Sorry ass game lmfao",1649577445903806464,marcelywelly,marcel 🥤,['1418928032937566218'],['drew😀'],['Zebandrew'],['1418928032937566218'],['Zebandrew'],0.14,0.19,0.06,0.15,0.04,0.38,0.04
3,1.885380e+18,1.885365e+18,twitter,"""@jensspahn Nein, die Bürgen wissen jetzt, das...",1825229241866141696,sofapolitik,Marcel Schneider,['299650387'],['Jens Spahn'],['jensspahn'],['299650387'],['jensspahn'],0.17,0.00,0.04,0.67,0.00,0.13,0.00
4,1.885379e+18,1.885301e+18,twitter,"""@AmberSmithUSA @stillgray Command an Autonomo...",1863320680747683840,marcelcous7,Marcel 🇮🇱 🇮🇱 🇮🇱,"['494140744', '16106584']","['Amber Smith', 'Ian Miles Cheong']","['AmberSmithUSA', 'stillgray']",['494140744'],['AmberSmithUSA'],0.04,0.01,0.00,0.91,0.02,0.01,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127545,1.882914e+18,1.882829e+18,twitter,"""@PM_ViktorOrban Why don’t Orban come to Khark...",1867778345858949120,nickerdoodle45,Nickerdoodle,['1577293480308654082'],['Orbán Viktor'],['PM_ViktorOrban'],['1577293480308654082'],['PM_ViktorOrban'],0.44,0.01,0.12,0.23,0.03,0.06,0.10
127546,1.882913e+18,1.882905e+18,twitter,"""@dior_milaa I was saying the exact same thing...",3232683137,AkhonaKi,BabyKim777,['1484921928989171714'],['ミ☆'],['dior_milaa'],['1484921928989171714'],['dior_milaa'],0.11,0.06,0.39,0.07,0.02,0.10,0.26
127547,1.885146e+18,1.884890e+18,twitter,"""@FrenchRapUS2 @FrenchRapUS Le seul son d'elle...",1264587955919822848,Marcelrod237,Marcel Rodrigue🇨🇲,"['1377649951669358599', '1120750848404000769']","['French Rap 🇺🇸', 'FRENCHRAPUS 🇺🇸']","['FrenchRapUS2', 'FrenchRapUS']",['1377649951669358599'],['FrenchRapUS2'],0.07,0.82,0.03,0.03,0.00,0.03,0.02
127548,1.885437e+18,1.884918e+18,twitter,"""@Marcel_Le_Gitan @BeyrutSpirit @Sans_drap__ A...",737114083,anne_lechevrel,Anne Lechevrel 🇨🇵 ✝️ ⚜️🎗🇷🇺,"['1789299094990602240', '1498228999998345220',...","['Marcel le Gitan', 'Beyrut_Spirit 🇮🇱🇨🇵💛💙💛💙💛💙'...","['Marcel_Le_Gitan', 'BeyrutSpirit', 'Sans_drap...",['1789299094990602240'],['Marcel_Le_Gitan'],0.34,0.01,0.14,0.24,0.00,0.27,0.00
